In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Dropout, LayerNormalization, Multiply, Reshape, Concatenate, Layer)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import RandomNormal, Orthogonal, Zeros
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from itertools import cycle
import random
import ast

In [2]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
print("GPUs detected:", gpus)


GPUs detected: []


In [3]:
df1=pd.read_csv('symtoms_df.csv')
df2=pd.read_csv('Symptom-severity.csv')

In [4]:
df1.head()

,Unnamed: 0,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4
0,0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches
1,1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN
2,2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN
3,3,Fungal infection,itching,skin_rash,dischromic _patches,NaN
4,4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN


In [5]:
df2.head()

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [6]:
print(df2.to_markdown())

|     | Symptom                        |   weight |
|----:|:-------------------------------|---------:|
|   0 | itching                        |        1 |
|   1 | skin_rash                      |        3 |
|   2 | nodal_skin_eruptions           |        4 |
|   3 | continuous_sneezing            |        4 |
|   4 | shivering                      |        5 |
|   5 | chills                         |        3 |
|   6 | joint_pain                     |        3 |
|   7 | stomach_pain                   |        5 |
|   8 | acidity                        |        3 |
|   9 | ulcers_on_tongue               |        4 |
|  10 | muscle_wasting                 |        3 |
|  11 | vomiting                       |        5 |
|  12 | burning_micturition            |        6 |
|  13 | spotting_urination             |        6 |
|  14 | fatigue                        |        4 |
|  15 | weight_gain                    |        3 |
|  16 | anxiety                        |        4 |
|  17 | cold

In [7]:
df1['Symptom_4'].fillna('',inplace=True)

C:\Users\Shashank\AppData\Local\Temp\ipykernel_23708\1593813174.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['Symptom_4'].fillna('',inplace=True)


In [8]:
df1['Symptoms']=df1['Symptom_1']+','+df1['Symptom_2']+','+df1['Symptom_3']+','+df1['Symptom_4']

In [9]:
df1=df1[['Symptoms','Disease']]

In [10]:
df1['Symptoms']=df1['Symptoms'].str.replace('_',' ')

In [11]:
df1.tail()

,Symptoms,Disease
4915,"vomiting, headache, nausea, spinning movements",(vertigo) Paroymsal Positional Vertigo
4916,"skin rash, pus filled pimples, blackheads, sc...",Acne
4917,"burning micturition, bladder discomfort, foul...",Urinary tract infection
4918,"skin rash, joint pain, skin peeling, silver l...",Psoriasis
4919,"skin rash, high fever, blister, red sore arou...",Impetigo


In [12]:
df2.tail()

,Symptom,weight
128,inflammatory_nails,2
129,blister,4
130,red_sore_around_nose,2
131,yellow_crust_ooze,3
132,prognosis,5


In [13]:
# Convert df2 into a dictionary: symptom -> weight
severity_dict = dict(zip(df2['Symptom'], df2['weight']))

In [14]:
# just want to check unique weights
set(severity_dict.values())

{1, 2, 3, 4, 5, 6, 7}

In [15]:
# --------------------------------------
# 3. Preprocess Symptoms from df1
# --------------------------------------
def preprocess_symptoms(symptom_str):
    """
    Converts comma-separated symptoms into a list of standardized tokens.
    1) Lowercase
    2) Trim spaces
    3) Replace inner spaces with underscores
    """
    # Split on commas
    symptoms = symptom_str.lower().split(',')
    # Clean each symptom token
    symptoms = [s.strip().replace(' ', '_') for s in symptoms]
    return symptoms

In [16]:
# Create a new column in df1 with the list of symptoms

df1['Symptom_list'] = df1['Symptoms'].apply(preprocess_symptoms)

In [17]:
df1.head()

,Symptoms,Disease,Symptom_list
0,"itching, skin rash, nodal skin eruptions, disc...",Fungal infection,"[itching, skin_rash, nodal_skin_eruptions, dis..."
1,"skin rash, nodal skin eruptions, dischromic ...",Fungal infection,"[skin_rash, nodal_skin_eruptions, dischromic__..."
2,"itching, nodal skin eruptions, dischromic pat...",Fungal infection,"[itching, nodal_skin_eruptions, dischromic__pa..."
3,"itching, skin rash, dischromic patches,",Fungal infection,"[itching, skin_rash, dischromic__patches, ]"
4,"itching, skin rash, nodal skin eruptions,",Fungal infection,"[itching, skin_rash, nodal_skin_eruptions, ]"


In [18]:
# ----------------------------------------------------
# 4. Tokenize the Symptoms (build a vocabulary)
# ----------------------------------------------------
# Flatten all symptom tokens to build a complete vocabulary

all_symptoms = [sym for row in df1['Symptom_list'] for sym in row]

tokenizer = Tokenizer(lower=True, filters='')  # no filters, since we've already cleaned
tokenizer.fit_on_texts(all_symptoms)


In [19]:
# ----------------------------------------------------
# 5. Convert Each List of Symptoms into Integer Sequences
# ----------------------------------------------------
# For each row, convert the list of symptom tokens into their integer IDs

df1['Symptom_seq'] = df1['Symptom_list'].apply(lambda x: tokenizer.texts_to_sequences(x))

# Flatten the list-of-lists for each row

df1['Symptom_seq'] = df1['Symptom_seq'].apply(lambda seq: [item for sublist in seq for item in sublist])


In [20]:
df1.head()

,Symptoms,Disease,Symptom_list,Symptom_seq
0,"itching, skin rash, nodal skin eruptions, disc...",Fungal infection,"[itching, skin_rash, nodal_skin_eruptions, dis...","[8, 5, 56, 57]"
1,"skin rash, nodal skin eruptions, dischromic ...",Fungal infection,"[skin_rash, nodal_skin_eruptions, dischromic__...","[5, 56, 57]"
2,"itching, nodal skin eruptions, dischromic pat...",Fungal infection,"[itching, nodal_skin_eruptions, dischromic__pa...","[8, 56, 57]"
3,"itching, skin rash, dischromic patches,",Fungal infection,"[itching, skin_rash, dischromic__patches, ]","[8, 5, 57]"
4,"itching, skin rash, nodal skin eruptions,",Fungal infection,"[itching, skin_rash, nodal_skin_eruptions, ]","[8, 5, 56]"


In [21]:
def shuffle_symptoms(row):
    """
    Takes a row with 'Symptoms', 'Symptom_list', and 'Symptom_seq'.
    Shuffles the symptom tokens (and corresponding token IDs) so that
    the mapping between symptom and token ID remains consistent.
    Returns new columns with the shuffled data.
    """
    # Pair each token with its corresponding token ID
    pairs = list(zip(row["Symptom_list"], row["Symptom_seq"]))

    # Shuffle the (token, token_id) pairs in place
    random.shuffle(pairs)

    # Unzip the shuffled pairs back into separate lists
    shuffled_symptoms, shuffled_seq = zip(*pairs)

    # Reconstruct the "Symptoms" string by joining shuffled tokens with commas
    shuffled_symptoms_str = ", ".join(shuffled_symptoms)

    # Return a Series containing the shuffled versions
    return pd.Series({
        "Shuffled_Symptoms": shuffled_symptoms_str,
        "Shuffled_Symptom_list": list(shuffled_symptoms),
        "Shuffled_Symptom_seq": list(shuffled_seq)
    })

In [22]:
# Apply the shuffle function row-by-row
df1[["Shuffled_Symptoms", "Shuffled_Symptom_list", "Shuffled_Symptom_seq"]] = df1.apply(shuffle_symptoms, axis=1)

# Display the Disease column as well as original and shuffled columns
df1[[
    "Symptoms", "Disease", "Symptom_list", "Symptom_seq",
    "Shuffled_Symptoms", "Shuffled_Symptom_list", "Shuffled_Symptom_seq"
]].head()
df1=df1.drop(columns={'Symptoms','Symptom_list','Symptom_seq'})

In [23]:
df1.head()

,Disease,Shuffled_Symptoms,Shuffled_Symptom_list,Shuffled_Symptom_seq
0,Fungal infection,"dischromic__patches, nodal_skin_eruptions, ski...","[dischromic__patches, nodal_skin_eruptions, sk...","[57, 56, 5, 8]"
1,Fungal infection,"dischromic__patches, nodal_skin_eruptions, ski...","[dischromic__patches, nodal_skin_eruptions, sk...","[57, 56, 5]"
2,Fungal infection,"nodal_skin_eruptions, dischromic__patches, itc...","[nodal_skin_eruptions, dischromic__patches, it...","[56, 57, 8]"
3,Fungal infection,"itching, skin_rash, dischromic__patches","[itching, skin_rash, dischromic__patches]","[8, 5, 57]"
4,Fungal infection,"skin_rash, nodal_skin_eruptions, itching","[skin_rash, nodal_skin_eruptions, itching]","[5, 56, 8]"


In [24]:
# ----------------------------------------------------
# 6. Pad the Sequences
# ----------------------------------------------------
max_length = max(df1['Shuffled_Symptom_seq'].apply(len))  # maximum length of any symptom sequence
X = pad_sequences(df1['Shuffled_Symptom_seq'], maxlen=max_length, padding='post', dtype='int32')



In [25]:
# ----------------------------------------------------
# 7. Encode Disease Labels
# ----------------------------------------------------
le = LabelEncoder()

y = le.fit_transform(df1['Disease'])

# Convert to one-hot vectors for a multi-class classification problem

y = tf.keras.utils.to_categorical(y)


In [26]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [27]:
# ----------------------------------------------------
# 8. Build a Weight Vector for the Vocabulary
# ----------------------------------------------------
# We'll create a numpy array, where each index corresponds to a token ID.
# If a token is found in severity_dict, use that weight; otherwise, default to 1.
vocab_size = len(tokenizer.word_index)
embedding_scaling = np.ones(vocab_size + 1, dtype=np.float32)  # +1 because index 0 is reserved (padding)

for token, index in tokenizer.word_index.items():
    if token in severity_dict:
        embedding_scaling[index] = severity_dict[token]
    else:
        embedding_scaling[index] = 1.0  # default severity weight

# Convert it to a constant tensor so it won't be trainable
embedding_scaling = tf.constant(embedding_scaling)


In [28]:
np.unique(embedding_scaling)

array([1., 2., 3., 4., 5., 6., 7.], dtype=float32)

In [29]:
df1=df1[['Shuffled_Symptoms','Shuffled_Symptom_list','Shuffled_Symptom_seq','Disease']]
df1.head()

,Shuffled_Symptoms,Shuffled_Symptom_list,Shuffled_Symptom_seq,Disease
0,"dischromic__patches, nodal_skin_eruptions, ski...","[dischromic__patches, nodal_skin_eruptions, sk...","[57, 56, 5, 8]",Fungal infection
1,"dischromic__patches, nodal_skin_eruptions, ski...","[dischromic__patches, nodal_skin_eruptions, sk...","[57, 56, 5]",Fungal infection
2,"nodal_skin_eruptions, dischromic__patches, itc...","[nodal_skin_eruptions, dischromic__patches, it...","[56, 57, 8]",Fungal infection
3,"itching, skin_rash, dischromic__patches","[itching, skin_rash, dischromic__patches]","[8, 5, 57]",Fungal infection
4,"skin_rash, nodal_skin_eruptions, itching","[skin_rash, nodal_skin_eruptions, itching]","[5, 56, 8]",Fungal infection


In [30]:
df2.head()

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [31]:
'''' Here you Need to Think of a New Approach How to deal with further  Shuffled_Symptom_seq and weights'''
'''one way is to put most weighted symptoms in last and least weighted symptoms first so that LSTM
   remembers High weighted one in the sequence in this weights also used and LSTM also works fine try this 
   Approach and see result
   in UI in First two selectbox user enters common symptoms and in last two most severe symptoms'''

'one way is to put most weighted symptoms in last and least weighted symptoms first so that LSTM\n   remembers High weighted one in the sequence in this weights also used and LSTM also works fine try this \n   Approach and see result\n   in UI in First two selectbox user enters common symptoms and in last two most severe symptoms'

In [32]:
''' Can we here think of New Model also despite LSTMs and RNNs --- Yes we use Some of the Attention Mechanism alse
    Rather than juss stick to RNN and LSTMs'''

' Can we here think of New Model also despite LSTMs and RNNs --- Yes we use Some of the Attention Mechanism alse\n    Rather than juss stick to RNN and LSTMs'

In [33]:
# Create a dictionary: {symptom: weight}
weight_map = dict(zip(df2["Symptom"], df2["weight"]))


In [34]:
weight_map

{'itching': 1,
 'skin_rash': 3,
 'nodal_skin_eruptions': 4,
 'continuous_sneezing': 4,
 'shivering': 5,
 'chills': 3,
 'joint_pain': 3,
 'stomach_pain': 5,
 'acidity': 3,
 'ulcers_on_tongue': 4,
 'muscle_wasting': 3,
 'vomiting': 5,
 'burning_micturition': 6,
 'spotting_urination': 6,
 'fatigue': 4,
 'weight_gain': 3,
 'anxiety': 4,
 'cold_hands_and_feets': 5,
 'mood_swings': 3,
 'weight_loss': 3,
 'restlessness': 5,
 'lethargy': 2,
 'patches_in_throat': 6,
 'irregular_sugar_level': 5,
 'cough': 4,
 'high_fever': 7,
 'sunken_eyes': 3,
 'breathlessness': 4,
 'sweating': 3,
 'dehydration': 4,
 'indigestion': 5,
 'headache': 3,
 'yellowish_skin': 3,
 'dark_urine': 4,
 'nausea': 5,
 'loss_of_appetite': 4,
 'pain_behind_the_eyes': 4,
 'back_pain': 3,
 'constipation': 4,
 'abdominal_pain': 4,
 'diarrhoea': 6,
 'mild_fever': 5,
 'yellow_urine': 4,
 'yellowing_of_eyes': 4,
 'acute_liver_failure': 6,
 'fluid_overload': 4,
 'swelling_of_stomach': 7,
 'swelled_lymph_nodes': 6,
 'malaise': 6,
 'bl

In [35]:
# For any symptom not in weight_map, use a default weight of 0
df1["Symptom_Weights"] = df1["Shuffled_Symptom_list"].apply(
    lambda sym_list: [weight_map.get(sym,1) for sym in sym_list]
)


In [36]:
df1.head()

,Shuffled_Symptoms,Shuffled_Symptom_list,Shuffled_Symptom_seq,Disease,Symptom_Weights
0,"dischromic__patches, nodal_skin_eruptions, ski...","[dischromic__patches, nodal_skin_eruptions, sk...","[57, 56, 5, 8]",Fungal infection,"[1, 4, 3, 1]"
1,"dischromic__patches, nodal_skin_eruptions, ski...","[dischromic__patches, nodal_skin_eruptions, sk...","[57, 56, 5]",Fungal infection,"[1, 4, 3]"
2,"nodal_skin_eruptions, dischromic__patches, itc...","[nodal_skin_eruptions, dischromic__patches, it...","[56, 57, 8]",Fungal infection,"[4, 1, 1]"
3,"itching, skin_rash, dischromic__patches","[itching, skin_rash, dischromic__patches]","[8, 5, 57]",Fungal infection,"[1, 3, 1]"
4,"skin_rash, nodal_skin_eruptions, itching","[skin_rash, nodal_skin_eruptions, itching]","[5, 56, 8]",Fungal infection,"[3, 4, 1]"


In [37]:
df2.weight.unique()

array([1, 3, 4, 5, 6, 2, 7], dtype=int64)

In [38]:
df1=df1[['Shuffled_Symptoms','Shuffled_Symptom_list','Symptom_Weights','Disease']]

In [39]:
df1.head()

,Shuffled_Symptoms,Shuffled_Symptom_list,Symptom_Weights,Disease
0,"dischromic__patches, nodal_skin_eruptions, ski...","[dischromic__patches, nodal_skin_eruptions, sk...","[1, 4, 3, 1]",Fungal infection
1,"dischromic__patches, nodal_skin_eruptions, ski...","[dischromic__patches, nodal_skin_eruptions, sk...","[1, 4, 3]",Fungal infection
2,"nodal_skin_eruptions, dischromic__patches, itc...","[nodal_skin_eruptions, dischromic__patches, it...","[4, 1, 1]",Fungal infection
3,"itching, skin_rash, dischromic__patches","[itching, skin_rash, dischromic__patches]","[1, 3, 1]",Fungal infection
4,"skin_rash, nodal_skin_eruptions, itching","[skin_rash, nodal_skin_eruptions, itching]","[3, 4, 1]",Fungal infection


In [40]:
df1.shape

(4920, 4)

In [41]:
df1[df1['Disease'].str.contains('Heart')].shape

(120, 4)

In [42]:
import ast

def safe_literal_eval(x):
    # If it's already a list, return it as-is.
    if isinstance(x, list):
        return x
    # Otherwise, try to evaluate the string representation
    try:
        return ast.literal_eval(x)
    except Exception as e:
        print(f"Error parsing {x}: {e}")
        return x

# Apply the safe evaluation function to the Symptom_Weights column.
df1['Symptom_Weights'] = df1['Symptom_Weights'].apply(safe_literal_eval)


In [43]:
df1['Symptom_Weights']

0       [1, 4, 3, 1]
1          [1, 4, 3]
2          [4, 1, 1]
3          [1, 3, 1]
4          [3, 4, 1]
            ...     
4915    [3, 6, 5, 5]
4916    [2, 2, 2, 3]
4917    [6, 4, 1, 6]
4918    [3, 2, 3, 3]
4919    [7, 4, 3, 2]
Name: Symptom_Weights, Length: 4920, dtype: object

In [44]:
# Process the symptoms: we assume they are a comma-separated string.
# If needed, you can further process (strip spaces, etc.)
df1['Symptom_Tokens'] = df1['Shuffled_Symptoms'].apply(lambda x: [sym.strip() for sym in x.split(',')])


In [45]:
# Determine maximum sequence length (number of symptoms per record)
max_len = df1['Symptom_Tokens'].apply(len).max()
max_len

4

In [46]:
# Prepare tokenizer on the symptom tokens
all_symptoms = df1['Symptom_Tokens'].tolist()
# Flatten the list of tokens
flat_symptoms = [sym for sublist in all_symptoms for sym in sublist]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(flat_symptoms)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

143


In [47]:
# Convert each record's symptom tokens to sequences of integers
df1['Symptom_Seq'] = df1['Symptom_Tokens'].apply(lambda tokens: tokenizer.texts_to_sequences(tokens))
# texts_to_sequences returns list of lists, so flatten each record
df1['Symptom_Seq'] = df1['Symptom_Seq'].apply(lambda seq_list: [item for sub in seq_list for item in sub])
# Pad sequences to uniform length (pad with zeros)
symptom_seqs = pad_sequences(df1['Symptom_Seq'], maxlen=max_len, padding='post')


In [48]:
symptom_seqs

array([[88,  4, 11, 13],
       [ 4, 88,  4, 11],
       [88, 86, 42, 13],
       ...,
       [ 5, 18, 43, 44],
       [ 4, 80, 10,  1],
       [81, 82, 83, 84]])

In [49]:
# Similarly, pad the weights. We assume weights are given in order.
# If a record has fewer weights than max_len, pad with zeros.
weights = np.array([np.pad(w, (0, max_len - len(w)), 'constant') for w in df1['Symptom_Weights']])
weights

array([[1, 4, 3, 1],
       [1, 4, 3, 0],
       [4, 1, 1, 0],
       ...,
       [6, 4, 1, 6],
       [3, 2, 3, 3],
       [7, 4, 3, 2]])

In [50]:
# Encode the disease labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df1['Disease'])
num_classes = len(label_encoder.classes_)
labels_cat = to_categorical(labels, num_classes=num_classes)


In [51]:
# Train-test split
X_symptoms_train, X_symptoms_test, X_weights_train, X_weights_test, y_train, y_test = train_test_split(
    symptom_seqs, weights, labels_cat, test_size=0.2, random_state=42
)

In [52]:
# -------------------------
# 2. Build the LSTM Model
# -------------------------
embedding_dim = 512  # Adjust embedding size as needed


In [53]:
# Input layers
symptoms_input = Input(shape=(max_len,), name="symptoms_input")
weights_input = Input(shape=(max_len,), name="weights_input")


In [54]:
# Embedding layer for symptoms
embedding_layer = Embedding(input_dim=vocab_size,
                            output_dim=embedding_dim,
                            input_length=max_len,
                            name="symptom_embedding")
embedded_symptoms = embedding_layer(symptoms_input)


In [55]:
# Reshape weights to multiply with embedding output
weights_reshaped = Reshape((max_len, 1), name="reshape_weights")(weights_input)
# Multiply embeddings by weights (element-wise multiplication)
weighted_embedding = Multiply(name="weighted_embedding")([embedded_symptoms, weights_reshaped])

In [56]:
weighted_embedding

<KerasTensor: shape=(None, 4, 512) dtype=float32 (created by layer 'weighted_embedding')>

In [57]:
# Reshape weights to match the dimensions of the embeddings (for element-wise multiplication)
weights_reshaped = Reshape((max_len, 1), name="reshape_weights")(weights_input)

# Multiply the embeddings by the corresponding weights
weighted_embedding = Multiply(name="weighted_embedding")([embedded_symptoms, weights_reshaped])

In [58]:
# LSTM layer with recurrent_dropout to force the fallback (non-cuDNN) implementation
lstm_out = LSTM(512, 
                activation='tanh', 
                recurrent_activation='sigmoid', 
                recurrent_dropout=0.2,  # This parameter forces the non-cuDNN kernel
                name="lstm_layer")(weighted_embedding)


In [59]:
# Add a dense layer for further processing
dense1 = Dense(64, activation='relu', name="dense_layer")(lstm_out)
# Output layer with softmax activation for multi-class classification
output = Dense(num_classes, activation='softmax', name="output_layer")(dense1)

In [60]:
# Build and compile the model
model = Model(inputs=[symptoms_input, weights_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 symptoms_input (InputLayer)    [(None, 4)]          0           []                               
                                                                                                  
 weights_input (InputLayer)     [(None, 4)]          0           []                               
                                                                                                  
 symptom_embedding (Embedding)  (None, 4, 512)       73216       ['symptoms_input[0][0]']         
                                                                                                  
 reshape_weights (Reshape)      (None, 4, 1)         0           ['weights_input[0][0]']          
                                                                                              

In [61]:
# -------------------------
# 3. Train the Model
# -------------------------
batch_size = 32
epochs = 20  # Increase epochs if necessary

history = model.fit(
    [X_symptoms_train, X_weights_train], y_train,
    validation_split=0.1,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1
)

Epoch 1/20
111/111 [==============================] - 11s 74ms/step - loss: 1.2368 - accuracy: 0.7120 - val_loss: 0.2788 - val_accuracy: 0.9137
Epoch 2/20
111/111 [==============================] - 8s 68ms/step - loss: 0.2321 - accuracy: 0.9212 - val_loss: 0.1959 - val_accuracy: 0.9492
Epoch 3/20
111/111 [==============================] - 8s 71ms/step - loss: 0.1860 - accuracy: 0.9297 - val_loss: 0.1878 - val_accuracy: 0.9442
Epoch 4/20
111/111 [==============================] - 8s 70ms/step - loss: 0.1576 - accuracy: 0.9373 - val_loss: 0.1658 - val_accuracy: 0.9442
Epoch 5/20
111/111 [==============================] - 8s 72ms/step - loss: 0.1392 - accuracy: 0.9449 - val_loss: 0.1435 - val_accuracy: 0.9518
Epoch 6/20
111/111 [==============================] - 7s 67ms/step - loss: 0.1344 - accuracy: 0.9438 - val_loss: 0.1762 - val_accuracy: 0.9391
Epoch 7/20
111/111 [==============================] - 8s 69ms/step - loss: 0.1300 - accuracy: 0.9458 - val_loss: 0.1538 - val_accuracy: 0.951

In [62]:
# -------------------------
# 4. Evaluate the Model
# -------------------------
loss, accuracy = model.evaluate([X_symptoms_test, X_weights_test], y_test, verbose=0)
print(f"Test Loss: {loss:.4f} | Test Accuracy: {accuracy:.4f}")


Test Loss: 0.1811 | Test Accuracy: 0.9268


In [63]:
# -------------------------
# 5. Making Predictions
# -------------------------
# For making predictions on new data, make sure to preprocess the symptom tokens and weights similarly.
# Example:
new_symptoms = "Chest_pain"
new_weights = [5, 6, 3]
new_tokens = [sym.strip() for sym in new_symptoms.split(',')]
new_seq = tokenizer.texts_to_sequences(new_tokens)
new_seq = [item for sub in new_seq for item in sub]
new_seq = pad_sequences([new_seq], maxlen=max_len, padding='post')
new_weights = np.array([np.pad(new_weights, (0, max_len - len(new_weights)), 'constant')])
prediction = model.predict([new_seq, new_weights])
predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
print("Predicted Disease:", predicted_label[0])


1/1 [==============================] - 0s 266ms/step
Predicted Disease: Heart attack


*********************************** Using Attention Layer *********************************************************